In [1]:
import pandas as pd
import json
filePath = r"C:\\Users\\Pat\\Desktop\\Projects\\LoLAramPredictor\\Game Info\\"

In [2]:
with open(filePath+'champion.json', encoding='utf8') as f:
    championData = json.load(f)

champions = pd.DataFrame(columns = ['championId', 'name', 'attack', 'defense', 'magic', 'difficulty', 'primaryClass', 'secondaryClass', 'partype', 'hp', 'hpperlevel', 'mp', 'mpperlevel', 'movespeed', 'armor', 'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel', 'mpregen', 'mpregenperlevel', 'crit', 'critperlevel', 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed'])

for k, v in championData['data'].items():
    champions = champions.append({'championId': int(v['key']),
                        'name': v['name'], 
                        'attack': v['info']['attack'], 
                        'defense': v['info']['defense'], 
                        'magic': v['info']['magic'], 
                        'difficulty': v['info']['difficulty'], 
                        'primaryClass': v['tags'][0], 
                        'secondaryClass': v['tags'][1] if len(v['tags']) > 1 else 'None', 
                        'partype': v['partype'], 
                        'hp': v['stats']['hp'], 
                        'hpperlevel': v['stats']['hpperlevel'], 
                        'mp': v['stats']['mp'], 
                        'mpperlevel': v['stats']['mpperlevel'], 
                        'movespeed': v['stats']['movespeed'], 
                        'armor': v['stats']['armor'], 
                        'armorperlevel': v['stats']['armorperlevel'], 
                        'spellblock': v['stats']['spellblock'], 
                        'spellblockperlevel': v['stats']['spellblockperlevel'], 
                        'attackrange': v['stats']['attackrange'], 
                        'hpregen': v['stats']['hpregen'], 
                        'hpregenperlevel': v['stats']['hpregenperlevel'], 
                        'mpregen': v['stats']['mpregen'], 
                        'mpregenperlevel': v['stats']['mpregenperlevel'], 
                        'crit': v['stats']['crit'], 
                        'critperlevel': v['stats']['critperlevel'], 
                        'attackdamage': v['stats']['attackdamage'], 
                        'attackdamageperlevel': v['stats']['attackdamageperlevel'], 
                        'attackspeedperlevel': v['stats']['attackspeedperlevel'], 
                        'attackspeed': v['stats']['attackspeed']}, ignore_index=True)

champions

,championId,name,attack,defense,magic,difficulty,primaryClass,secondaryClass,partype,hp,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,266,Aatrox,8,4,3,4,Fighter,Tank,Blood Well,580,...,3,1,0,0,0,0,60,5,2.500,0.651
1,103,Ahri,3,4,8,5,Mage,Assassin,Mana,526,...,6.5,0.6,8,0.8,0,0,53.04,3,2.000,0.668
2,84,Akali,5,3,8,7,Assassin,None,Energy,575,...,8,0.5,50,0,0,0,62.4,3.3,3.200,0.625
3,12,Alistar,6,9,5,7,Tank,Support,Mana,600,...,8.5,0.85,8.5,0.8,0,0,62,3.75,2.125,0.625
4,32,Amumu,2,6,8,3,Tank,Mage,Mana,613.12,...,9,0.85,7.382,0.525,0,0,53.38,3.8,2.180,0.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,238,Zed,9,2,1,7,Assassin,None,Energy,584,...,7,0.65,50,0,0,0,63,3.4,3.300,0.651
146,115,Ziggs,2,4,9,4,Mage,None,Mana,536,...,6.5,0.6,8,0.8,0,0,54.208,3.1,2.000,0.656
147,26,Zilean,2,5,8,6,Support,Mage,Mana,504,...,5.5,0.5,11.335,0.8,0,0,51.64,3,2.130,0.625
148,142,Zoe,1,7,8,5,Mage,Support,Mana,560,...,6.5,0.6,8,0.65,0,0,58,3.3,2.500,0.625


In [3]:
import asyncpg
import nest_asyncio
import asyncio
from config import dbConfig

nest_asyncio.apply()
loop = asyncio.get_event_loop()

In [4]:
import itertools
champNameCombo = list(itertools.combinations(champions['name'], 2))
print(champNameCombo[0])

champIdCombinations = list(itertools.combinations(champions['championId'], 2))
# champIdCombinations

('Aatrox', 'Ahri')


In [5]:
async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
    stmt = await con.prepare(query)
    columns = [a.name for a in stmt.get_attributes()]
    data = await stmt.fetch()
    return pd.DataFrame(data, columns=columns)

async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        allGamesQuery = f'SELECT "gameId", "championId", "teamColor", "win" \
                    FROM "Participant_Stats" \
                    ORDER BY "gameId" ASC;'
        allGames_df = await fetch_as_dataframe(connection, allGamesQuery)

    return allGames_df

allGames_df = loop.run_until_complete(run())
allGames_df.head(10)

,gameId,championId,teamColor,win
0,2855142709,22,Blue,False
1,2855142709,498,Red,True
2,2855142709,21,Blue,False
3,2855142709,122,Blue,False
4,2855142709,75,Blue,False
5,2855142709,20,Blue,False
6,2855142709,57,Red,True
7,2855142709,117,Red,True
8,2855142709,267,Red,True
9,2855142709,29,Red,True


In [6]:
allGames_df['championId'] = pd.to_numeric(allGames_df['championId'], downcast='unsigned')
allGames_df['teamColor'] = allGames_df['teamColor'] == 'Red'
allGames_df.memory_usage(deep=True)

Index              128
gameId        19764920
championId     4941230
teamColor      2470615
win            2470615
dtype: int64

In [7]:
from multiprocessing.pool import ThreadPool as Pool
from statsmodels.stats.proportion import proportion_confint

In [20]:
def calcWR(champ):
    allGames_df['isSynergy'] = allGames_df['championId'].isin(champ)
    synergyGames_df = allGames_df.loc[allGames_df['isSynergy'] == True, ['gameId', 'teamColor', 'win']]
    synergyGames_df = synergyGames_df.groupby(['gameId', 'teamColor']).agg(['sum', 'count'])
    synergyGames_df = synergyGames_df.loc[synergyGames_df['win']['count'] == 2, 'win']
    CI = proportion_confint(synergyGames_df['sum'].astype(bool).sum(axis=0), len(synergyGames_df.index), method='binom_test', alpha=0.05)
    return {'champion1': champions.loc[champions["championId"] == champ[0], "name"].values[0],
                                            'champion2': champions.loc[champions["championId"] == champ[1], "name"].values[0], 
                                            'winRate': synergyGames_df['sum'].astype(bool).sum(axis=0)/len(synergyGames_df.index),
                                            '95% CI': f"[{CI[0]:.6f}, {CI[1]:.6f}]",
                                            'numWins': synergyGames_df['sum'].astype(bool).sum(axis=0),
                                            'numGames': len(synergyGames_df.index)
                                            }

if __name__ == '__main__':
    pool = Pool(6)
    res = pool.map(calcWR, champIdCombinations)
    twoSynergy_df = pd.DataFrame.from_dict(res)
    twoSynergy_df


In [21]:
twoSynergy_df.sort_values(axis=0, ascending=False, by='winRate', inplace=True)
twoSynergy_df.head(10)

,champion1,champion2,winRate,95% CI,numWins,numGames
3501,Elise,Sett,0.678261,"[0.587196, 0.758075]",78,115
5058,Heimerdinger,Xerath,0.658537,"[0.613160, 0.701927]",297,451
10702,Taric,Yorick,0.658537,"[0.500000, 0.794812]",27,41
7681,Lissandra,Yorick,0.657534,"[0.541280, 0.761771]",48,73
5066,Heimerdinger,Ziggs,0.653030,"[0.615958, 0.688722]",431,660
7127,Kled,Sion,0.652174,"[0.529154, 0.755782]",45,69
11150,Yorick,Ziggs,0.651429,"[0.577283, 0.720456]",114,175
8282,Miss Fortune,Sion,0.649485,"[0.612714, 0.684915]",441,679
6391,Karthus,Teemo,0.648956,"[0.619577, 0.677470]",684,1054
1883,Brand,Heimerdinger,0.648190,"[0.615989, 0.679361]",573,884


In [22]:
twoSynergy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11175 entries, 3501 to 11030
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   champion1  11175 non-null  object 
 1   champion2  11175 non-null  object 
 2   winRate    11175 non-null  float64
 3   95% CI     11175 non-null  object 
 4   numWins    11175 non-null  int64  
 5   numGames   11175 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 611.1+ KB


In [24]:
twoSynergy_df.to_csv('twoSynergy.csv', sep=',')